In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 19:29:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/12 19:30:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
from pyspark.sql import Row
data = [Row(seat_id=1, free=1),
        Row(seat_id=2, free=0),
        Row(seat_id=3, free=1),
        Row(seat_id=4, free=1),
        Row(seat_id=5, free=1)]

# Create DataFrame
df = spark.createDataFrame(data)

# Show the DataFrame
df.show()

print(df.dtypes)

+-------+----+
|seat_id|free|
+-------+----+
|      1|   1|
|      2|   0|
|      3|   1|
|      4|   1|
|      5|   1|
+-------+----+

[('seat_id', 'bigint'), ('free', 'bigint')]


In [4]:
df.createOrReplaceTempView("cinema")

In [5]:
result = spark.sql(
    """
    -- wrong solution, missing the lass seat_id
    Select seat_id 
    From (Select *, lead(free) over(order by seat_id) as next from cinema ) r
    Where free = 1 and next = 1

    """
)
result.show()

23/12/12 12:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+
|seat_id|
+-------+
|      3|
|      4|
+-------+



In [6]:
result = spark.sql(
    """          
    -- wrong solution, missing the lass seat_id
    Select seat_id 
    From (Select *, lead(free) over(order by seat_id) as next, lag(free) over(order by seat_id) as prev from cinema ) r
    Where (free = 1 and next = 1) or(free = 1 and prev = 1)
    """
)
result.show()

23/12/12 12:55:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:55:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:55:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+
|seat_id|
+-------+
|      3|
|      4|
|      5|
+-------+



23/12/12 12:55:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/12 12:55:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [15]:
result = spark.sql(
    """          
    -- duplicates
    Select distinct a.seat_id
    From cinema a join cinema b on (a.seat_id + 1 = b.seat_id) or a.seat_id = b.seat_id + 1
    Where (a.free = 1 and b.free = 1) 
    Order by a.seat_id

    """
)
result.show()

+-------+
|seat_id|
+-------+
|      3|
|      4|
|      5|
+-------+



In [16]:
result = spark.sql(
    """          
    Select distinct a.seat_id
    From cinema a join cinema b on abs(a.seat_id - b.seat_id) = 1
    Where (a.free = 1 and b.free = 1) 
    Order by a.seat_id

    """
)
result.show()

+-------+
|seat_id|
+-------+
|      3|
|      4|
|      5|
+-------+



In [1]:
rows = [
    { 'month': 'jan', 'customerId': '101', 'sales': 105 },
    { 'month': 'feb', 'customerId': '101', 'sales': 106 },
    { 'month': 'feb', 'customerId': '101', 'sales': 200 },
    { 'month': 'jan', 'customerId': '100', 'sales': 50 },
    { 'month': 'feb', 'customerId': '100', 'sales': 100 }
]

[{'month': 'jan', 'customerId': '101', 'sales': 105}, {'month': 'feb', 'customerId': '101', 'sales': 106}, {'month': 'feb', 'customerId': '101', 'sales': 200}, {'month': 'jan', 'customerId': '100', 'sales': 50}, {'month': 'feb', 'customerId': '100', 'sales': 100}]


In [2]:
result = {}
for row in rows:
    if row['sales'] >= 100:
        result[row['month']] = result.get(row['month'], set())
        result[row['month']].add(row['customerId'])

print({k:len(v) for k,v in result.items()})

{'jan': 1, 'feb': 2}


In [6]:
from pyspark.sql import Row
data = [Row(**row) for row in rows]

# Create DataFrame
df = spark.createDataFrame(data)

# Show the DataFrame
df.show()

print(df.dtypes)
df.createOrReplaceTempView("rows")

+-----+----------+-----+
|month|customerId|sales|
+-----+----------+-----+
|  jan|       101|  105|
|  feb|       101|  106|
|  feb|       101|  200|
|  jan|       100|   50|
|  feb|       100|  100|
+-----+----------+-----+

[('month', 'string'), ('customerId', 'string'), ('sales', 'bigint')]


In [8]:
result = spark.sql(
    """          
    SELECT
    month,
    COUNT(DISTINCT(customerId)) as count
FROM
    rows
WHERE sales >= 100
GROUP BY month;

    """
)
result.show()

+-----+-----+
|month|count|
+-----+-----+
|  feb|    2|
|  jan|    1|
+-----+-----+



In [10]:
result = spark.sql(
    """          
    SELECT
    month,
    COUNT(DISTINCT(case when sales >= 100 then customerId else null end)) as count
    FROM rows
    GROUP BY month;

    """
)
result.show()

+-----+-----+
|month|count|
+-----+-----+
|  feb|    2|
|  jan|    1|
+-----+-----+

